In [1]:
import aocd
from aocd.models import Puzzle
day = 19
year = 2020
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [2]:
data = puzzle.input_data.splitlines()
data[:10]

['132: 80 20 | 46 54',
 '107: 58 20 | 3 54',
 '55: 126 20 | 107 54',
 '93: 110 29',
 '8: 42',
 '75: 92 20 | 103 54',
 '113: 91 20 | 35 54',
 '94: 54 64 | 20 21',
 '46: 54 20 | 110 54',
 '26: 54 74 | 20 94']

In [201]:
test_data  = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
"""

In [47]:
from functools import lru_cache

In [127]:
def parse(data):
    rules = []
    messages = []
    parse = 'rules'
    for line in data:
        if line.strip() == "":
            parse = "messages"
            continue
        if parse == 'rules':
            rules.append(parse_rules(line))
        else:
            messages.append(line)
    return rules, messages

def parse_rules(line):
    num_rule, rules = line.split(': ')
    if rules in ("\"a\"", "\"b\""):
        return (int(num_rule), rules[1])
#     print(num_rule, rules)
    rules = tuple([tuple(map(int, rule.split(' '))) for rule in rules.split(' | ')])
#     return rules.split(' | ')
    return (int(num_rule), rules)

In [137]:
# rules, messages = parse(test_data.splitlines())
rules, messages = parse(puzzle.input_data.splitlines())
rules = dict(rules)

In [138]:
rules

{132: ((80, 20), (46, 54)),
 107: ((58, 20), (3, 54)),
 55: ((126, 20), (107, 54)),
 93: ((110, 29),),
 8: ((42,),),
 75: ((92, 20), (103, 54)),
 113: ((91, 20), (35, 54)),
 94: ((54, 64), (20, 21)),
 46: ((54, 20), (110, 54)),
 26: ((54, 74), (20, 94)),
 61: ((20, 76), (54, 25)),
 128: ((38, 20), (29, 54)),
 129: ((20, 81), (54, 49)),
 21: ((20, 49), (54, 9)),
 103: ((54, 2), (20, 51)),
 9: ((54, 54), (20, 20)),
 82: ((54, 9), (20, 29)),
 127: ((20, 54), (54, 54)),
 59: ((40, 54), (87, 20)),
 0: ((8, 11),),
 105: ((20, 125), (54, 1)),
 78: ((82, 54), (132, 20)),
 88: ((20, 63), (54, 13)),
 53: ((20, 46), (54, 38)),
 62: ((54, 123), (20, 65)),
 116: ((20, 19), (54, 66)),
 32: ((118, 20), (26, 54)),
 81: ((54, 54),),
 45: ((54, 50), (20, 98)),
 15: ((124, 54), (73, 20)),
 118: ((59, 54), (85, 20)),
 64: ((49, 20), (13, 54)),
 10: ((20, 95), (54, 128)),
 95: ((54, 80), (20, 81)),
 11: ((42, 31),),
 97: ((20, 46),),
 108: ((20, 77), (54, 88)),
 63: ((110, 110),),
 125: ((29, 54), (12, 20)

In [148]:
@lru_cache(maxsize=None)
def get_length(nr):
    rule = rules[nr]
    if rule in ('a', 'b'):
        return 1
    if len(rule) == 1:
        return sum([get_length(r) for r in rule[0]])
    if len(rule) == 2:
        left = sum([get_length(r) for r in rule[0]])
        right = sum([get_length(r) for r in rule[1]])
        if left == right:
            return left
        raise
    raise
get_length.cache_clear()

In [140]:
@lru_cache(maxsize=None)
def check_set(line, rs):
    i = 0
    for r in rs:
        l = get_length(r)
        if not check(line[i:l+i],r):
            return False
        i+=l
    return True

@lru_cache(maxsize=None)
def check(line, nr):
#     print(line, nr)
    rule = rules[nr]
    l = get_length(nr)
    if len(line) != l:
        return False
    if rule in ('a', 'b'):
        return line == rule
    if len(rule) == 1:
#         print(rule[0], line)
        return check_set(line, rule[0])
    if len(rule) == 2:
        return check_set(line, rule[0]) or check_set(line, rule[1])
    raise
    
check_set.cache_clear()
check.cache_clear()

In [141]:
valid = sum([check(m, 0) for m in messages])

118

In [142]:
puzzle.answer_a = valid

That's the right answer!  You are one gold star closer to saving your vacation. [Continue to Part Two]


In [143]:
# Part B

In [163]:
@lru_cache(maxsize=None)
def get_length_B(nr):
    rule = rules[nr]
    if rule in ('a', 'b'):
        return 1
    if len(rule) == 1:
        return sum([get_length_B(r) for r in rule[0]])
    if len(rule) == 2:
        left = sum([get_length_B(r) for r in rule[0]])
        if nr in (8, 11):
            print('!')
            return left
        right = sum([get_length_B(r) for r in rule[1]])
        if left == right:
            return left
        raise
    raise
get_length.cache_clear()

@lru_cache(maxsize=None)
def check_set_B(line, rs):
    i = 0
    for r in rs:
        l = get_length_B(r)
        if not check(line[i:l+i],r):
            return False
        i+=l
    return True

@lru_cache(maxsize=None)
def check_B(line, nr):
#     print(line, nr)
    rule = rules[nr]
    l = get_length_B(nr)
    if len(line) < l:
        return False
    if rule in ('a', 'b'):
        return line == rule
    if len(rule) == 1:
#         print(rule[0], line)
        return check_set_B(line, rule[0])
    if len(rule) == 2:
        return check_set_B(line, rule[0]) or check_set_B(line, rule[1])
    raise

In [165]:
get_length_B(31)

8

In [187]:
# another try with regex

In [188]:
import re

In [309]:
def get_match_string(nr):
    rule = rules[nr]
    if rule in ('a', 'b'):
        return rule
    if len(rule) == 1:
        left = "".join([get_match_string(r) for r in rule[0]])
        return "{}".format(left)
    if len(rule) == 2:
        left = "".join([get_match_string(r) for r in rule[0]])
        right = "".join([get_match_string(r) for r in rule[1]])
        return "(({})|({}))".format(left, right)
    raise
    
def get_match_string_B(nr):
    rule = rules[nr]
    if rule in ('a', 'b'):
        return rule
    if nr == 8:        
        return "({})+".format(get_match_string_B(42))
    if nr == 11:
        ret = "("
        for i in range(1, 10):
            ret += "((%s){%d}(%s){%d})" % (get_match_string_B(42), i, get_match_string_B(31), i)
            if i != 9:
                ret += "|"
#             ret += "(({})\{1\}({}))".format(get_match_string_B(42), get_match_string_B(31))
        return ret + ")"
#         return "(({})+({})+)".format(get_match_string_B(42), get_match_string_B(31))
    if len(rule) == 1:
        left = "".join([get_match_string_B(r) for r in rule[0]])
        return "{}".format(left)
    if len(rule) == 2:
        left = "".join([get_match_string_B(r) for r in rule[0]])
        right = "".join([get_match_string_B(r) for r in rule[1]])
        return "(({})|({}))".format(left, right)
    raise

In [310]:
# rules, messages = parse(test_data.splitlines())
rules, messages = parse(puzzle.input_data.splitlines())
rules = dict(rules)

In [286]:
match_string = get_match_string(0)+"$"

pattern = re.compile(match_string)
sum(list(map(lambda m: re.match(pattern, m) is not None, messages )))

118

In [287]:
test_data_B = """42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba"""

In [311]:
# rules, messages = parse(test_data_B.splitlines())
rules, messages = parse(puzzle.input_data.splitlines())
rules = dict(rules)

In [312]:
match_string = get_match_string_B(0)+"$"

pattern = re.compile(match_string)
sum(list(map(lambda m: re.match(pattern, m) is not None, messages )))

246

In [294]:
puzzle.answer_b = _

1183